## Wide & Deep NN

In [8]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
assert tf.__version__ >= "2.0"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

import pandas as pd

2.13.0


In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [14]:
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
X_new = X_test[:3]

In [17]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [18]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 8)]                  0         []                            
                                                                                                  
 dense_7 (Dense)             (None, 30)                   270       ['input_2[0][0]']             
                                                                                                  
 dense_8 (Dense)             (None, 30)                   930       ['dense_7[0][0]']             
                                                                                                  
 concatenate_1 (Concatenate  (None, 38)                   0         ['input_2[0][0]',             
 )                                                                   'dense_8[0][0]']       

In [19]:
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
y_pred = model.predict(X_new)

Epoch 1/20
363/363 [==============================] - 0s 574us/step - loss: 1.9430 - val_loss: 0.9191
Epoch 2/20
363/363 [==============================] - 0s 444us/step - loss: 0.8670 - val_loss: 0.7768
Epoch 3/20
363/363 [==============================] - 0s 430us/step - loss: 0.7465 - val_loss: 0.6985
Epoch 4/20
363/363 [==============================] - 0s 454us/step - loss: 0.6730 - val_loss: 0.6586
Epoch 5/20
363/363 [==============================] - 0s 452us/step - loss: 0.6340 - val_loss: 0.6272
Epoch 6/20
363/363 [==============================] - 0s 474us/step - loss: 0.6044 - val_loss: 0.6047
Epoch 7/20
363/363 [==============================] - 0s 461us/step - loss: 0.5819 - val_loss: 0.5827
Epoch 8/20
363/363 [==============================] - 0s 446us/step - loss: 0.5628 - val_loss: 0.5672
Epoch 9/20
363/363 [==============================] - 0s 448us/step - loss: 0.5484 - val_loss: 0.5543
Epoch 10/20
363/363 [==============================] - 0s 432us/step - loss: 0.536

In [20]:
y_pred

array([[0.7401556],
       [1.4516754],
       [2.7926033]], dtype=float32)

와이드나 딥 경로에 다른 입력 특성을 전달하면 어떻게 될까요? (특성 0에서 4까지) 5개의 특성을 와이드 경로에 보내고 (특성 2에서 7까지) 6개의 특성을 딥 경로에 전달하겠습니다. 3개의 특성(특성 2, 3, 4)은 양쪽에 모두 전달됩니다.

In [23]:
np.random.seed(42)
tf.random.set_seed(42)

In [25]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [26]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
# history = model.fit({"wide_input": X_train_A, "deep_input": X_train_B}, y_train, epochs=20, validation_data=({"wide_input": X_valid_A, "deep_input": X_valid_B}, y_valid))
# 딕셔너리 형태로 명시해줄 수 있음

mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Epoch 1/20
363/363 [==============================] - 0s 618us/step - loss: 1.9893 - val_loss: 0.9051
Epoch 2/20
363/363 [==============================] - 0s 442us/step - loss: 0.7966 - val_loss: 0.7065
Epoch 3/20
363/363 [==============================] - 0s 453us/step - loss: 0.6626 - val_loss: 0.6378
Epoch 4/20
363/363 [==============================] - 0s 654us/step - loss: 0.6091 - val_loss: 0.6033
Epoch 5/20
363/363 [==============================] - 0s 454us/step - loss: 0.5792 - val_loss: 0.5793
Epoch 6/20
363/363 [==============================] - 0s 455us/step - loss: 0.5597 - val_loss: 0.5671
Epoch 7/20
363/363 [==============================] - 0s 541us/step - loss: 0.5459 - val_loss: 0.5515
Epoch 8/20
363/363 [==============================] - 0s 508us/step - loss: 0.5349 - val_loss: 0.5418
Epoch 9/20
363/363 [==============================] - 0s 462us/step - loss: 0.5265 - val_loss: 0.5352
Epoch 10/20
363/363 [==============================] - 0s 447us/step - loss: 0.520

In [27]:
y_pred

array([[0.61095685],
       [1.7675967 ],
       [2.980628  ]], dtype=float32)

### 출력이 여러 개 필요할 때
- 동일한 데이터에서 독립적인 여러 작업을 수행할 때
- 규제 기법으로 사용하는 경우

In [28]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])

In [32]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-3))

In [33]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20,
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Epoch 1/20
363/363 [==============================] - 1s 846us/step - loss: 2.6756 - main_output_loss: 2.4743 - aux_output_loss: 4.4875 - val_loss: 1.2355 - val_main_output_loss: 0.9947 - val_aux_output_loss: 3.4023
Epoch 2/20
363/363 [==============================] - 0s 528us/step - loss: 0.9995 - main_output_loss: 0.8040 - aux_output_loss: 2.7587 - val_loss: 0.8584 - val_main_output_loss: 0.7007 - val_aux_output_loss: 2.2770
Epoch 3/20
363/363 [==============================] - 0s 517us/step - loss: 0.8018 - main_output_loss: 0.6715 - aux_output_loss: 1.9742 - val_loss: 0.7600 - val_main_output_loss: 0.6484 - val_aux_output_loss: 1.7638
Epoch 4/20
363/363 [==============================] - 0s 526us/step - loss: 0.7283 - main_output_loss: 0.6313 - aux_output_loss: 1.6012 - val_loss: 0.7103 - val_main_output_loss: 0.6239 - val_aux_output_loss: 1.4881
Epoch 5/20
363/363 [==============================] - 0s 532us/step - loss: 0.6871 - main_output_loss: 0.6087 - aux_output_loss: 1.3925 

In [34]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

162/162 [==============================] - 0s 396us/step - loss: 0.5469 - main_output_loss: 0.5145 - aux_output_loss: 0.8379


1/1 [==============================] - 0s 26ms/step


In [37]:
print(y_pred_main)
print()
print(y_pred_aux)

[[0.7054903]
 [1.759553 ]
 [2.7266278]]

[[1.1412153]
 [1.844955 ]
 [1.9123092]]
